## Feature Selection

- Create scikit learn basic pipeline
- Standardize the train data (Done in 8)
- Options :
    - Regularization (glmnet) (Lasso, Ridge, Elastic Net)
    - PCA , MCA
    - Trees - Feature Importance (Random Forest - with filter, Decision Tree)
    - VIF testing
    - Forward Selection
    - Backward Selection    
    - Recursive Feature Elimination
    - Feature Importance (Random Forest, Decision Tree)
    - Correlation Matrix
    


idea - make MCA with GEOIDS - UNiv occurence?

## Read Data, Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, Lasso
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from statistics import mean
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import RFE

boxplots, scatterplots, scatterplots for categoricals to see imteraction, cat by num, color code by women involvement

In [2]:
os.getcwd()

'C:\\Users\\trent\\Documents\\GitHub\\patents22'

In [3]:
os.chdir('C:\\Users\\trent\\Documents\\Capstone')

In [4]:
train_read = pd.read_csv('train_standardized.csv')

C:\Users\trent\AppData\Local\Temp\ipykernel_17260\4173701639.py:1: DtypeWarning: Columns (0,6,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  train_read = pd.read_csv('train_standardized.csv')


### Check for Null Values, Remove Columns with Null Values from Model Building

In [5]:
for i in range(len(train_read.columns)):
    if train_read.iloc[:,i].isnull().values.any() == True:
        print(train_read.iloc[:,i].name,': ',train_read.iloc[:,i].isna().sum())

patent_num :  585876
assignee :  4253
grant_yr :  585876
ipc :  681113
assignee_univ_map :  4253
Institution :  1528381
Control :  1528381
Award_Amount :  68919
Award_by_pop_gt_16_lf :  68919
ipc_A :  681113
ipc_B :  681113
ipc_C :  681113
ipc_D :  681113
ipc_E :  681113
ipc_F :  681113
ipc_G :  681113
ipc_H :  681113
num_ipcs :  681113


In [6]:
##drop columns not necessary for model
##dropping ipc and award data as well for now to create model
train = train_read.drop(['patent_num',
                   'assignee',
                   'app_num',
                   'GEOID',
                   'assignee_univ_map',
                   'Institution',
                   'Control',
                   'grant_yr',
                    'Award_Amount',
                    'Award_by_pop_gt_16_lf',
                    'ipc_A',
                    'ipc_B',
                    'ipc_C',
                    'ipc_D',
                    'ipc_E',
                    'ipc_F',
                    'ipc_G',
                    'ipc_H',
                    'num_ipcs',
                    'ipc'],
                   axis = 1)

In [7]:
for i in range(len(train.columns)):
    if train.iloc[:,i].isnull().values.any() == True:
        print(train.iloc[:,i].name,': ',train.iloc[:,i].isna().sum())

In [8]:
numeric = [
 # 'num_ipcs',
 'team_size',
 'men_inventors',
 'women_inventors',
 'Over25_Less9Grade',
 'Over25_SomeHS',
 'Over25_HSGrad',
 'Over25_SomeCollege',
 'Over25_Assosc',
 'Over25_Bach',
 'Over25_Grad',
 'ap_by_est',
 'pop_gt_16_lf',
 'est_by_pop_gt_16_lf',
 'Total_employment',
 'Pop_Est',
 'Population_Estimate',
 'Population_persons',
 'Earnings_by_place_of_work',
 'Employee_and_self-employed_contributions_for_government_social_insurance',
 'Employer_contributions_for_employee_pension_and_insurance_funds',
 'Employer_contributions_for_government_social_insurance',
 'Equals_Net_earnings_by_place_of_residence',
 'Farm_income',
 'Farm_proprietors_income',
 'Less_Contributions_for_government_social_insurance',
 'Nonfarm_personal_income',
 'Nonfarm_proprietors_income',
 'Per_capita_personal_income_dollars',
 'Personal_income_thousands_of_dollars',
 'Plus_Adjustment_for_residence',
 'Plus_Dividends_interest_and_rent',
 'Plus_Personal_current_transfer_receipts',	
 'Proprietors_employment',
 'Proprietors_income',
 'Supplements_to_wages_and_salaries',
 'Wage_and_salary_employment',
 'Wages_and_salaries',
 'Applications',
 # 'Award_Amount',
 # 'GDP_by_labor_force',
 'apps_by_pop_gt_16_lf',
 'Wages_by_pop_gt_16_lf',
 # 'Award_by_pop_gt_16_lf'
]

In [9]:
categorical = [
# 'ipc',
# 'ipc_A',
# 'ipc_B',
# 'ipc_C',
# 'ipc_D',
# 'ipc_E',
# 'ipc_F',
# 'ipc_G',
# 'ipc_H',
'already_granted',
# 'Control',
'r1',
'r2',
'special_focus',
'bea_region',
'Agriculture_Forestry_Fishing_Hunting_base',
'Mining_Quarrying_and_Oil_Gas_Extraction_base',
'Utilities_base',
'Construction_base',
'Manufacturing_base',
'Wholesale_Trade_base',
'Retail_Trade_base',
'Transportation_Warehousing_base',
'Information_base',
'Finance_Insurance_base',
'Real_Estate_Rental_Leasing_base',
'Professional_Scientific_and_Technical_Services_base',
'Management_of_Companies_Enterprises_base',
'Administrative_Support_Waste_Management_Remediation_Services_base',
'Educational_Services_base',
'Health_Care_Social_Assistance_base',
'Arts_Entertainment_and_Recreation_base',
'Accommodation_Food_Services_base',
'Other_Services_except_Public_Administration_base',
'women_involved'
]

### Cleaning Data for Models - One Hot Encode Categoricals
All already binary except region

In [10]:
train_val = pd.concat([train.drop('bea_region', axis =1), pd.get_dummies(train.bea_region)], axis = 1)

In [57]:
# from sklearn.compose import ColumnTransformer

In [58]:
# categorical_preprocessor = OneHotEncoder(handle_unknown = "ignore")

In [67]:
# preprocessor = ColumnTransformer([
#     ('one-hot-encoder', categorical_preprocessor, train[categorical]),
#     # ('standard_scaler', numerical_preprocessor, numerical_columns)
#     ])

In [11]:
y_wi= train_val['women_involved']
x_wi= train_val.drop('women_involved', axis = 1)
y_gdp= train_val['GDP_by_labor_force']
x_gdp= train_val.drop('GDP_by_labor_force', axis = 1)

In [42]:
# def prepare_inputs(X_train, X_test):
#     oe = OrdinalEncoder()
#     oe.fit(X_train)
#     X_train_enc = oe.transform(X_train)
#     X_test_enc = oe.transform(X_test)
#     return X_train_enc, X_test_enc


In [47]:
# from sklearn.model_selection import train_test_split

## Feature Selection/Importance Methods in Python

### Categorical Features - Women Involvement

#### Simple Logistic Regression Model

In [19]:
model = LogisticRegression(solver='liblinear')
logreg = model.fit(x_wi, y_wi)
# evaluate the model
# yhat = model.predict(x_wi)
# # evaluate predictions
# accuracy = accuracy_score(y_test, yhat)
# print('Accuracy: %.2f' % (accuracy*100))

In [21]:
logreg.coef_

array([[-7.86903231e-08,  5.41366178e-11,  1.03225140e-11,
         1.31245052e-10, -2.60719018e-11,  5.99446715e-13,
        -9.74442846e-14,  1.23838127e-14,  2.50829602e-11,
         1.56102189e-12, -2.33150192e-11, -1.50469977e-11,
        -1.42742982e-11,  1.14845735e-11,  1.53335616e-11,
        -1.92068628e-11, -1.24553736e-11, -1.22406778e-11,
        -3.90438227e-11, -4.03379483e-11, -3.87214373e-11,
        -3.97790808e-11, -3.48937764e-11, -3.67481154e-11,
        -4.05670413e-11, -3.59668590e-11, -4.03831928e-11,
        -4.28370745e-11, -2.96634485e-11, -3.36170606e-11,
        -3.95672411e-11, -3.92291392e-11, -3.81201771e-11,
        -4.02298349e-11,  1.00952425e-08, -3.34159459e-08,
        -3.61188938e-08, -3.68994924e-12, -1.83361137e-12,
        -5.28944019e-12, -1.74888055e-11, -1.65587944e-11,
        -1.80119275e-11, -2.04500412e-11, -1.00586245e-11,
        -1.35638877e-11, -1.75028318e-11, -1.55886992e-11,
        -1.68853866e-11, -1.90652313e-11, -6.84490793e-1

#### Chi Squared Test
Test for independence between categorical variables. The larger the score the more relevant the feature is.

##### First filtering for the latest categoricals after one hot encoding:

In [26]:
categorical = [
# 'ipc',
# 'ipc_A',
# 'ipc_B',
# 'ipc_C',
# 'ipc_D',
# 'ipc_E',
# 'ipc_F',
# 'ipc_G',
# 'ipc_H',
'already_granted',
# 'Control',
'r1',
'r2',
'special_focus',
# 'bea_region',
'Agriculture_Forestry_Fishing_Hunting_base',
'Mining_Quarrying_and_Oil_Gas_Extraction_base',
'Utilities_base',
'Construction_base',
'Manufacturing_base',
'Wholesale_Trade_base',
'Retail_Trade_base',
'Transportation_Warehousing_base',
'Information_base',
'Finance_Insurance_base',
'Real_Estate_Rental_Leasing_base',
'Professional_Scientific_and_Technical_Services_base',
'Management_of_Companies_Enterprises_base',
'Administrative_Support_Waste_Management_Remediation_Services_base',
'Educational_Services_base',
'Health_Care_Social_Assistance_base',
'Arts_Entertainment_and_Recreation_base',
'Accommodation_Food_Services_base',
'Other_Services_except_Public_Administration_base',
# 'women_involved'
'Far West',
'Great Lakes', 
'Mideast', 
'New England', 
'Plains', 
'Rocky Mountain',
'Southeast', 
'Southwest'
]

In [28]:
# x_wi[categorical]

In [32]:
fs = SelectKBest(score_func=chi2, k='all')
fs.fit(x_wi[categorical], y_wi)
X_train_fs = fs.transform(x_wi[categorical])

In [40]:
for i in range(len(fs.scores_)):
    print(fs.feature_names_in_[i],'Feature %d: %f' % (i, fs.scores_[i]))
    # plot the scores
    # plt.bar([i for i in range(len(fs.scores_))], fs.scores_)
    # plt.show()


already_granted Feature 0: 366.432474
r1 Feature 1: 8191.994005
r2 Feature 2: 132.741806
special_focus Feature 3: 508.178829
Agriculture_Forestry_Fishing_Hunting_base Feature 4: 39.593571
Mining_Quarrying_and_Oil_Gas_Extraction_base Feature 5: 1260.784565
Utilities_base Feature 6: 2.330940
Construction_base Feature 7: 1.396460
Manufacturing_base Feature 8: 2572.124821
Wholesale_Trade_base Feature 9: 31.762102
Retail_Trade_base Feature 10: 883.294073
Transportation_Warehousing_base Feature 11: 293.423892
Information_base Feature 12: 228.659141
Finance_Insurance_base Feature 13: 358.016253
Real_Estate_Rental_Leasing_base Feature 14: 292.717175
Professional_Scientific_and_Technical_Services_base Feature 15: 476.822535
Management_of_Companies_Enterprises_base Feature 16: 289.368435
Administrative_Support_Waste_Management_Remediation_Services_base Feature 17: 1608.998544
Educational_Services_base Feature 18: 821.693058
Health_Care_Social_Assistance_base Feature 19: 141.765695
Arts_Entertain

#### Recursive Feature Elimination

In [ ]:
rfe = RFE(estimator = DecisionTreeClassifier(), n_features_to_select = 10)
model = DecisionTreeClassifier()
pipeline = Pipeline(steps=[('s',rfe),('m',model)])
cv = RepeatedStratifiedKFold(n_splites = 10, n_repeats=3, random_state=1)
n_scores = cross_val_score(pipeline, x_wi, y_wi, scoring = 'accuracy', cv = cv, n_jobs=-1)


### Numerical - Women Involvement

#### Anova F Test

In [45]:
fs = SelectKBest(k='all')
fs.fit(x_wi[numeric], y_wi)
X_train_fs = fs.transform(x_wi[numeric])

for i in range(len(fs.scores_)):
    print(fs.feature_names_in_[i],'Feature %d: %f' % (i, fs.scores_[i]))
    # plot the scores
    # plt.bar([i for i in range(len(fs.scores_))], fs.scores_)
    # plt.show()

team_size Feature 0: 217239.557043
men_inventors Feature 1: 6413.022931
women_inventors Feature 2: 4313000.261570
Over25_Less9Grade Feature 3: 0.002827
Over25_SomeHS Feature 4: 2310.601329
Over25_HSGrad Feature 5: 2530.228534
Over25_SomeCollege Feature 6: 7890.419521
Over25_Assosc Feature 7: 3708.346035
Over25_Bach Feature 8: 3261.616918
Over25_Grad Feature 9: 8036.418915
ap_by_est Feature 10: 2055.820773
pop_gt_16_lf Feature 11: 74.420270
est_by_pop_gt_16_lf Feature 12: 1942.412615
Total_employment Feature 13: 1.617859
Pop_Est Feature 14: 119.823406
Population_Estimate Feature 15: 124.223970
Population_persons Feature 16: 124.198317
Earnings_by_place_of_work Feature 17: 151.496958
Employee_and_self-employed_contributions_for_government_social_insurance Feature 18: 134.262198
Employer_contributions_for_employee_pension_and_insurance_funds Feature 19: 143.760705
Employer_contributions_for_government_social_insurance Feature 20: 183.650467
Equals_Net_earnings_by_place_of_residence Featur

Tuning number of selected features:

In [12]:
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

In [13]:
# define the pipeline to evaluate
model = LogisticRegression(solver='liblinear')
fs = SelectKBest()
pipeline = Pipeline(steps=[('anova',fs), ('lr', model)])

In [16]:
grid = dict()
grid['anova__k'] = [i+1 for i in range(5)]

In [17]:
search = GridSearchCV(pipeline, grid, scoring='accuracy', n_jobs=-1, cv=cv)
# perform the search
results = search.fit(x_wi[numeric], y_wi)

#### Linear Regression

In [27]:
# testing numerical only
x_gdp_num = x_gdp[numeric]
linear_GDP = LinearRegression()
linear_GDP.fit(x_gdp_num, y_gdp)

LinearRegression()

In [30]:
linear_GDP.summary()

AttributeError: 'LinearRegression' object has no attribute 'summary'

### Regularization

#### Ridge

In [41]:
# model = Ridge(alpha = 1.0)
cv = RepeatedKFold(n_splits = 10, n_repeats = 3, random_state=1)
model = RidgeCV(alphas=np.arange(0.01,1,0.01), 
                cv=cv,
                scoring = 'neg_mean_absolute_error')
model.fit(x_gdp,y_gdp)


                         

KeyboardInterrupt: 

## Pipeline

In [ ]:

# estimators = 
model = make_pipeline(preprocessor, LogisticRegression(max_iter = 500))
model